In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from lac.utils.camera import Camera
from lac.utils.plotting import plot_surface, plot_poses, plot_3d_points
from lac.util import load_data

%load_ext autoreload
%autoreload 2

In [ ]:
# Generate a set of z=f(x,y) points according to sin(x)*sin(y)
n = 100
x = np.linspace(0, 2 * np.pi, n)
y = np.linspace(0, 2 * np.pi, n)
X, Y = np.meshgrid(x, y)
Z = np.cos(X) * np.cos(Y)
color = Z.flatten()
grid = np.stack([X, Y, Z], axis=-1)
points = np.stack([X.flatten(), Y.flatten(), Z.flatten()], axis=-1)

plot_surface(grid)

In [ ]:
camera_t = np.array([0, 3, 1])
camera_R = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]).T
camera_pose = np.eye(4)
camera_pose[:3, :3] = camera_R
camera_pose[:3, 3] = camera_t

cam = Camera(camera_pose)

In [ ]:
fig = plot_surface(grid)
fig = plot_poses([camera_pose], fig=fig)
fig.show()

In [ ]:
uv_inframe, depths_inframe, color_inframe = cam.project_world_points_to_uv(points, color)

In [ ]:
# Sort the points by depth
sorted_indices = np.argsort(depths_inframe)
uv_sorted = uv_inframe[sorted_indices]
depths_sorted = depths_inframe[sorted_indices]
color_sorted = color_inframe[sorted_indices]

In [ ]:
c = plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c=depths_sorted[::-1], cmap="plasma")
plt.colorbar(c)
# Set x and y limits
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
blank_image = np.zeros((720, 1280)) * np.nan
pixel_set = np.zeros_like(blank_image, dtype=bool)

pt_size_u = 20
pt_size_v = 2

start_time = time.perf_counter()
for idx in sorted_indices:
    u, v = uv_inframe[idx]

    if pixel_set[int(v), int(u)]:
        continue
    # Color a 5x5 patch around the point
    for u_i in range(int(u - pt_size_u / 2), int(u + pt_size_u / 2)):
        for v_j in range(int(v - pt_size_v / 2), 720):
            if 0 <= u_i < 1280 and 0 <= v_j < 720:
                if not pixel_set[v_j, u_i]:
                    blank_image[int(v_j), int(u_i)] = color_inframe[idx]
                    pixel_set[v_j, u_i] = True
print("Time taken:", time.perf_counter() - start_time)


fig = plt.figure(figsize=(20, 10))
c = plt.imshow(blank_image, cmap="plasma")
plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c="black", alpha=0.1, cmap="plasma")
plt.colorbar(c)
plt.show()

In [ ]:
uv, depths, colors = cam.project_world_points_to_uv(points, color)

## Load map


In [ ]:
from lac.mapping.mapper import interpolate_heights
from lac.utils.frames import get_cam_pose_rover

from scipy.interpolate import griddata

In [ ]:
map = np.load("../../../data/heightmaps/qualifying/Moon_Map_01_preset_0.dat", allow_pickle=True)
X_map, Y_map, Z_map = map[..., 0], map[..., 1], map[..., 2]
map_points = np.stack([X_map.flatten(), Y_map.flatten(), Z_map.flatten()], axis=-1)

data_path = "../../../output/LocalizationAgent/map1_preset0_4m_spiral"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
plot_surface(map)

In [ ]:
plot_3d_points(map_points, color=Z_map.ravel())

In [ ]:
B = 4.425
INTERP_RES = 0.05
x_interp = np.arange(-B, B, INTERP_RES)
y_interp = np.arange(-B, B, INTERP_RES)
X_interp, Y_interp = np.meshgrid(x_interp, y_interp)

# Interpolate with griddata
Z_interp = griddata(
    (X_map.ravel(), Y_map.ravel()),
    Z_map.ravel(),
    (X_interp.ravel(), Y_interp.ravel()),
    method="linear",
)
Z_interp = Z_interp.reshape(X_interp.shape)

In [ ]:
map_interp = np.stack([X_interp, Y_interp, Z_interp], axis=-1)
map_interp_points = np.stack([X_interp.ravel(), Y_interp.ravel(), Z_interp.ravel()], axis=-1)
plot_surface(map_interp)

In [ ]:
plot_3d_points(map_interp_points, color=Z_interp.ravel())

In [ ]:
from lac.utils.frames import CAMERA_TO_OPENCV_PASSIVE

In [ ]:
rover_pose = poses[6500]
camera_pose = get_cam_pose_rover("FrontLeft")
camera_pose[:3, :3] = CAMERA_TO_OPENCV_PASSIVE
camera_pose = rover_pose @ camera_pose

In [ ]:
# camera_t = np.array([0, 0, 1.8])
# camera_R = np.array([[0, -1, 0], [0, 0, -1], [1, 0, 0]]).T
# camera_pose = np.eye(4)
# camera_pose[:3, :3] = camera_R
# camera_pose[:3, 3] = camera_t

cam = Camera(camera_pose)

fig = plot_surface(map)
fig = plot_poses([camera_pose, rover_pose], fig=fig)
fig.show()

In [ ]:
uv, depths, colors = cam.project_world_points_to_uv(map_interp_points, Z_interp.flatten())

In [ ]:
# Sort the points by depth
sorted_indices = np.argsort(depths)
uv_sorted = uv[sorted_indices]
depths_sorted = depths[sorted_indices]
colors_sorted = colors[sorted_indices]

In [ ]:
fig = plt.figure(figsize=(15, 7))
# c = plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c=depths_sorted[::-1], cmap="plasma")
c = plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c=colors_sorted[::-1], cmap="plasma")
plt.colorbar(c)
# Set x and y limits
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
from lac.params import FL_X

blank_image = np.zeros((720, 1280)) * np.nan
pixel_set = np.zeros_like(blank_image, dtype=bool)

pt_size_u = 100
pt_size_v = 2
# CELL_WIDTH_M = 0.15
CELL_WIDTH_M = INTERP_RES

start_time = time.perf_counter()
for idx in sorted_indices:
    u, v = uv[idx]
    depth = depths[idx]

    if pixel_set[int(v), int(u)]:
        continue
    # Color a 5x5 patch around the point
    pt_size_u = (CELL_WIDTH_M * FL_X) / depth
    for u_i in range(int(u - pt_size_u / 2), int(u + pt_size_u / 2)):
        for v_j in range(int(v - pt_size_v / 2), 720):
            if 0 <= u_i < 1280 and 0 <= v_j < 720:
                if not pixel_set[v_j, u_i]:
                    blank_image[int(v_j), int(u_i)] = colors_sorted[idx]
                    pixel_set[v_j, u_i] = True
print("Time taken:", time.perf_counter() - start_time)


fig = plt.figure(figsize=(20, 10))
c = plt.imshow(blank_image, cmap="plasma")
# plt.scatter(uv_sorted[:, 0][::-1], uv_sorted[:, 1][::-1], c="black", alpha=0.1, cmap="plasma")
plt.colorbar(c)
plt.show()